In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
from oirpy.oirreader import Oirreader
from bactinfection import segmentation, automated
import matplotlib.pyplot as plt
import skimage.io
import numpy as np

from pathlib import Path
import re
import os

import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster


In [ ]:
#if __name__ == '__main__':

In [ ]:
client = Client()#'tcp://127.0.0.1:55963')

#cluster = SLURMCluster(cores=1, memory='4GB', walltime='3:00:00')
#client = Client(cluster)
#cluster.scale(jobs=20)

### Specify the main folder to analyze and the main folder where to store analyzed data

In [ ]:
#data_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Leticia')
#save_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Results_Leticia3/')
#nucl_model_type = 'nuclei'
#diameter_nucl = 60
#nucl_channel='DAPI'
#cell_channel='Alexa Fluor 633'
#masking='cell_no_nuclei'

data_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Stefano')
save_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Results_Stefano')
nucl_model_type = 'cyto'
diameter_nucl = 90
nucl_channel='Alexa Fluor 633'
cell_channel=None
masking='nuclei'

#data_folder_base = Path('/storage/research/mic_rs/Oevermann/Listeria_Image_Analysis/Leticia')
#save_folder_base = Path('/storage/research/mic_rs/Oevermann/Results_Leticia_actin')


### Find all directories containing oir files 

In [ ]:
all_dirs = []
for dirName, subdirList, fileList in os.walk(data_folder_base):
    if len(list(Path(dirName).glob('*.oir'))) > 0:
        all_dirs.append(dirName)

### Go through all directories and run the full analysis on each image

Each single analysis is created as a dask.delayed object and added to a list. The full computation is effectively executed after that with ```dask.compute()```.

In [ ]:
all_tasks = []
for a in all_dirs[0:2]:
    foldername = Path(a)
    save_folder = save_folder_base.joinpath(foldername.relative_to(data_folder_base))
    if not save_folder.is_dir():
        save_folder.mkdir(parents=True)
    all_files = [x for x in foldername.glob("*.oir")]
    if (masking == 'cell') or (masking == 'cell_no_nuclei'):
        all_files = [x for x in all_files if save_folder.joinpath(x.name.replace('.oir', '_cell_seg.tif')).is_file()]
    
    if len(all_files) == 0:
        continue
        
    if all_files[0].name.lower().find('macro') > 0:
        diameter_cell = 140
    else:
        diameter_cell = 300
        
    for f in all_files[0::5]:
        task = dask.delayed(automated.single_image_analysis)(
        filepath=f,
        diameter_nucl=diameter_nucl,
        diameter_cell=diameter_cell,
        save_folder=save_folder,
        nucl_channel=nucl_channel,
        cell_channel=cell_channel,
        bact_channel='Alexa Fluor 488',
        actin_channel='Alexa Fluor 633',
        bact_width=5,
        bact_len=5,
        corr_threshold=0.5,
        min_corr_vol=3,
        n_std=10,
        nucl_model_type=nucl_model_type,
        masking=masking
        )
        all_tasks.append(task)

In [ ]:
res = dask.compute(all_tasks)